# Python aplicado: Bolsa de valores

# Funcionamento do mercado de ações

O produto conhecido como ação é nada mais que a fração de uma empresa, ou seja, quando você adquire uma ação você está de certa forma tornando-se sócio da empresa, podendo ganhar dinheiro com a valorização e o lucro dela. 
De forma geral, as ações são negociadas na chamada bolsa de valores, e cada páis tem a sua própria bolsa, podendo inclusive, ter mais de uma. No Brasil a bolsa de valores está sediada em São Paulo e é chamada de B3.
Para negociar cada ação geralmente usa-se o que é chamado de corretora, sendo através dela que uma ordem de compra ou de venda é enviada para a bolsa de valores. 
Conforme as ordens de compras e vendas são executadas ao longo dos dias, semanas e meses, um histórico de negociação vai sendo formado. É com base nesse histórico que é possível avaliar o desempenho de uma ação ou papel. A obtenção deste histórico pode ser feita de várias formas, incluíndo obtenção de dados em tempo real. Para fins de estudos, iremos trabalhar com uma base de dados históricos e não em tempo real.

# Instalação do ambiente de desenvolvimento

Neste curso iremos trabalhar com 3 bibliotecas principais:
<br>
- pandas: biblioteca para de manipulação de estrutura de dados 
- pandas-datareader: responsável por buscar nossos dados históricos e converte-los em um formato que podemos trabalhar com pandas
- plotly: biblioteca para plotagem dos dados. Nencessário gerar um chave no site https://plot.ly/python/

### Criando um ambiente virtual

Primeira vamos criar um ambiente virtual, com o venv, nativo do python3:

python -m venv python-finance

cd python-finance

Scripts\activate

mkdir lessons

cd lessons

# Treino

In [ ]:
import pandas_datareader as pdr
import datetime 

vale3 = pdr.get_data_yahoo('VALE3.SA', 
                          start=datetime.datetime(2016, 10, 1), 
                          end=datetime.datetime(2019, 1, 1))

# analyszng the dataset 

# get first 5 rows
display(vale3.head())

# get last 5 rows
display(vale3.tail())

# get statistcs of dataset 
display(vale3.describe())

In [ ]:
# Inspect the index 
print(vale3.index)

# Inspect the columns
print(vale3.columns)

In [ ]:
vale3['2018-01-01':].head(10)

In [ ]:
pct_c = (vale3.Close / vale3.Open).apply(lambda x: x - 1) * 100
display(pct_c.head(10))

In [ ]:
import numpy as np

# Assign `Adj Close` to `daily_close`
daily_close = vale3.Close

# Daily returns
daily_pct_change = daily_close.pct_change()

# Replace NA values with 0
daily_pct_change.fillna(0, inplace=True)

# Inspect daily returns
display(daily_pct_change.head(10))

# Daily log returns
daily_log_returns = np.log(daily_close.pct_change()+1)

# Print daily log returns
display(daily_log_returns.head(10))

In [ ]:
# Daily log returns
daily_log_returns = np.log(vale3.Close.pct_change()+1)

In [ ]:
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np

plotly_username = ""
plotly_key = ""

# auth to plotly 
py.sign_in(plotly_username, plotly_key)

In [ ]:

data = [go.Histogram(x=daily_log_returns)]
py.iplot(data, filename='basic histogram')

# Obtendo os dados

Nossos dados serão obtidos atravás da biblioteca pandas-datareader. Vamos testar com 3 ativos (ações):

In [ ]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
import pandas_datareader as pdr
import plotly.graph_objs as go

Autenticando no serviço do plotly

In [ ]:
plotly_username = ""
plotly_key = ""

# auth to plotly 
py.sign_in(plotly_username, plotly_key)

Funções para retornar os dados

In [ ]:
# we download the stock prices for each ticker and then we do a mapping between data and name of the ticker
def get(tickers, startdate, enddate):
  def data(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map (data, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))


Download dos dados

In [ ]:
# we download the stock prices for each ticker and then we do a mapping between data and name of the ticker
def get(tickers, startdate, enddate):
  def data(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map (data, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))


Download dos dados

In [ ]:
# Define the stocks to download. We'll download of Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', '^GSPC']

# We would like all available data from 01/01/2000 until 31/12/2018.
start_date = datetime.datetime(2016, 1, 1)
end_date = datetime.datetime(2019, 12, 31)

all_data = get(tickers, start_date, end_date)

In [ ]:
all_data.head(10)

In [ ]:
all_data.tail(10)

Calculando o ganho da carteira de maneira simplista

In [ ]:
# transform rows in columns
daily_close_px = all_data[['Adj Close']].reset_index().pivot('Date', 'Ticker', 'Adj Close')

# get the actual price and split by previous price to find the price variation
cart_pct_change = daily_close_px / daily_close_px.shift(1) - 1

Também podemos verificar o valor acumulado diariamente, através da função cumprod, que retorna o valor acumulado da lista, neste exemplo, o percentual acumulado

In [ ]:
# sum all prices variation percentages
cart_return = (1 + cart_pct_change).cumprod()

cart_return.head(10)

Por fim, para verificarmos o rendimento final da carteira, basta visualizar o último valor da lista de produtos acumulados

In [ ]:
# percentual values
final_return = cart_return[-1:].apply(lambda x: (x - 1) *100)
display(final_return)

# Plotando Candlestick

In [ ]:
trace = go.Candlestick(x=all_data.loc['MSFT'].index,
                open=all_data.loc['MSFT'].Open,
                high=all_data.loc['MSFT'].High,
                low=all_data.loc['MSFT'].Low,
                close=all_data.loc['MSFT'].Close)
data = [trace]
py.iplot(data)

# Preparando os dados

Primemiro reogarnizamos a estrutura do dataset para facilitar a manipulação dos dados e pegamos o preço de fechamento

In [ ]:
# rearrange data
data = all_data.reset_index()
data = data.set_index(['Date','Ticker']).sort_index()

# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
close = data['Close']

Descobrimos quais foram os dias úteis no período dentro do range de data que selecionamos anteriormente. Então reindexamos nosso dataset com esse novo range de data. Isto fará com que data inexistentes sejam criadas com o valores nulos (NaN)

In [ ]:
# Getting all weekdays between 01/01/2000 and 31/12/2018
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex close using all_weekdays as the new index
close = close.reindex( pd.MultiIndex.from_product([all_weekdays,  tickers], names=['Date', 'Ticker']),fill_value=np.NaN)
#close = close.fillna(0)
close.head(10)

Por fim, iremos transformar a coluna ticker em 3 colunas distintas, uma para cada ativo.

In [ ]:
close = close.reset_index().pivot(index="Date", columns="Ticker", values="Close")

Agora nosso dataset está pronto e podemos dar uma olhlada nele.

In [ ]:
close.describe()

Calcular o  alor percentual utilizando a função pct_change

In [ ]:
data = all_data.pct_change(1)
data.loc['MSFT'].Close.head()

É o equivalente ao que fizemos no cálculo do ganho da carteira

In [ ]:
(daily_close_px / daily_close_px.shift(1) - 1)['MSFT'].head()

Como iremos trabalhar na forma logaritimica, iremos usar uma outra estratégia para cálculo do percentual. Primeiro aplicamos o logaritimo no valor do número e depois calculamos a diferança através função diff, que calcula a diferença dos valores ao longo de uma lista (out[n] = a[n+1] - a[n])

In [ ]:
log_returns = np.log(close.dropna()).diff()
log_returns.head()

Além do preço relativo ou percentual da ação, muitas vezes também se trabalha com o total relativo acumalado, que é a medida que indica a rentabibilidade de um dado período.
Os gráfico abaixo representamo o valorp percentual da ação seguido pelo gráfico representando o total acumulado. Por exemplo, no gráfico de total acumulado, notamos que se uma pessoa tivesse  investido 1000 na Apple no inicio dos anos 2000, teria aumentado seu capital em quase 6 vezes.

In [ ]:
layout = go.Layout(
    title='Result',
    yaxis=dict(
        title='Cumulative log returns',
        titlefont=dict(
            size=26,
            color='#7f7f7f'
        )
    )
)
    
axis = []

for d in log_returns:
    axi = go.Scatter(
                x=log_returns.index,
                y=log_returns[d].cumsum(),
                name = d,
                opacity = 1)
    axis.append(axi)
    
fig = dict(data=axis, layout=layout)
py.iplot(fig)

In [ ]:
layout = go.Layout(
    title='Result',
    yaxis=dict(
        title='Total relative returns (%)',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
axis = []

for d in log_returns:
    axi = go.Scatter(
                x=log_returns.index,
                y= 100 * (np.exp(log_returns[d].cumsum()) -1),
                name = d,
                opacity = 1)
    axis.append(axi)
    
fig = dict(data=axis, layout=layout)
py.iplot(fig)

# Buy and Hold
Vamos assumir que iremos cálcular o rendimemnto do último dia disponível na nossa carteira. A forma mais simples de realizar este cálculo e multiplicando a matriz de pesos pela matriz de preços da ação neste dia. Primeiro devemos transpor a matriz dos preços

In [ ]:
r_t = log_returns.tail(1).transpose()
r_t

Em seguida criar a matriz dos pesos das ações dentro da carteira. Aqui utilizamos 1/3, pois cada uma das três ações possuem o mesmo peso dentro da carteira.

In [ ]:
weights_vector = pd.DataFrame(1 / 3, index=r_t.index, columns=r_t.columns)
weights_vector

Realizamos a multiplicação e temos o rendimento da carteira como um todo

In [ ]:
portfolio_log_return = weights_vector.transpose().dot(r_t)
portfolio_log_return

A fim de calcular para todo o período de tempo

In [ ]:
weights_matrix = pd.DataFrame(1 / 3, index=log_returns.index, columns=log_returns.columns)
weights_matrix.head()

In [ ]:
temp_var = weights_matrix.dot(log_returns.transpose())
temp_var.tail()

O valor que procuramos está na diganoal da matriz, pois veja que é quando as datas das matrizes de origem se interseccionam. Para estrar este valor utilizamos a função diag, do numpy

In [ ]:
portfolio_log_returns = pd.Series(np.diag(temp_var), index=log_returns.index)
portfolio_log_returns = portfolio_log_returns[1:]
portfolio_log_returns.tail()

A plotagem dos gráficos são similares ao que já fizemos anteriormente

In [ ]:
layout = go.Layout(
    title='Result',
    yaxis=dict(
        title='Portfolio cumulative log returns',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
    
axis = go.Scatter(
            x=portfolio_log_returns.index,
            y=portfolio_log_returns.cumsum(),
            opacity = 1)
    
fig = dict(data=[axis], layout=layout)
py.iplot(fig)

In [ ]:
total_relative_returns = (np.exp(portfolio_log_returns.cumsum()) - 1)

layout = go.Layout(
    title='Result',
    yaxis=dict(
        title='Portfolio total relative returns (%)',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
    
axis = go.Scatter(
            x = total_relative_returns.index,
            y = total_relative_returns * 100,
            opacity = 1)
    
fig = dict(data=[axis], layout=layout)
py.iplot(fig)

# Análise de trading por médias móveis

## Média móvel simples

In [ ]:
# Get the MSFT timeseries. This now returns a Pandas Series object indexed by date and drop NA rows.
msft = all_data.loc['MSFT'].dropna()
msft = msft['2018-06-01':]

window = 21
# Calculate the  21 days moving averages of the closing prices
MA = msft.Close.rolling(window=window).mean()



trace_avg = go.Scatter(
                x=MA.index,
                y=MA,
                name = "MSFT MA(21)",
                line = dict(color = '#BEBECF'),
                opacity = 1)

trace_candles = go.Candlestick(x=msft.index,
                    open=msft.Open,
                    high=msft.High,
                    low=msft.Low,
                    close=msft.Close,
                    name = "Price")

data = [trace_avg, trace_candles]

fig = dict(data=data)
py.iplot(fig)

## Média móvel exponencial

In [ ]:
window = 21
K = ( 2 / (window + 1))

msft = all_data.loc['MSFT'].dropna()
msft = msft['2018-06-01':]

MA = msft.Close.rolling(window=window).mean().dropna()


In [ ]:
ema_data = pd.DataFrame(index=MA.index)
ema_data['Price'] = msft.Close
ema_data['MA'] = MA
ema_data['EMA'] = np.NaN

ema_data.EMA[0] = ema_data.MA[1]

for i in range(1, len(ema_data)):
    ema_data.EMA[i] = (ema_data.Price[i] * K) + ((1 - K) * ema_data.EMA[i-1]) 

In [ ]:
trace_ma = go.Scatter(
                x=ema_data.index,
                y=ema_data.MA,
                name = "MSFT MA(21)",
                line = dict(color = '#BEBECF'),
                opacity = 1)

trace_ema = go.Scatter(
                x=ema_data.index,
                y=ema_data.EMA,
                name = "MSFT EMA(21)",
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_candles = go.Candlestick(x=msft.index,
                    open=msft.Open,
                    high=msft.High,
                    low=msft.Low,
                    close=msft.Close,
                    name = "Price")

data = [trace_ma, trace_ema, trace_candles]

fig = dict(data=data)
py.iplot(fig)

# Cruzamento de Médias
Calcular as duas médias móveis, de 9 e 21 períodos

In [ ]:
# Get the MSFT timeseries. This now returns a Pandas Series object indexed by date and drop NA rows.
msft = close.loc[:, 'MSFT'].dropna()

# Calculate the 9 and 21 days moving averages of the closing prices
short_rolling_msft = msft.rolling(window=9).mean()
long_rolling_msft = msft.rolling(window=21).mean()

A função rolling foi utilizada em conjunto com a função mean(). Neste cenário estamos calculando as média dos valores selecionadas na nossa "janela" de tempo. Poderiamos utilizar outras funções, como sum() para somá-los ou std() para cálcular o desvio padrão.

Para plotagem do gráfico, iremos utilziar o plotly. Começamos por criar os 3 eixos necessários: eixo da média curta, eixo da média longa, eixo do preço.

In [ ]:
trace_short = go.Scatter(
                x=short_rolling_msft.index,
                y=short_rolling_msft,
                name = "MSFT Short",
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_long = go.Scatter(
                x=long_rolling_msft.index,
                y=long_rolling_msft,
                name = "MSFT Long",
                line = dict(color = '#7F7F7F'),
                opacity = 1)

trace_price = go.Scatter(
                x=msft.index,
                y=msft,
                name = "MSFT Price",
                line = dict(color = '#B22222'),
                opacity = 0.8)

data = [trace_short, trace_long, trace_price]

Então invocamos a função de plotagem

In [ ]:
fig = dict(data=data)
py.iplot(fig)

# MACD

In [ ]:
from plotly import tools

msft = all_data.loc['MSFT'].dropna()
msft = msft['2018-06-01':]

In [ ]:
def get_ema(window, prices):
    K = ( 2 / (window + 1))
    ma = prices.rolling(window=window).mean().dropna()

    data = pd.DataFrame(index=ma.index)
    data['Price'] = prices
    data['EMA'] = np.NaN

    data.EMA[0] = ma[1]

    for i in range(1, len(data)):
        data.EMA[i] = (data.Price[i] * K) + ((1 - K) * data.EMA[i-1]) 

    return data

In [ ]:
mm_12 = get_ema(12, msft.Close)
mm_26 = get_ema(26, msft.Close)

mm_macd = mm_12.EMA - mm_26.EMA
mm_signal = get_ema(9, mm_macd.dropna()).EMA

In [ ]:
msft = msft[mm_signal.index[0]:]

trace_macd = go.Scatter(
                    x=mm_macd.index,
                    y=mm_macd,
                    name = "MACD",
                    line = dict(color = '#17BECF'),
                    opacity = 1)

trace_signal = go.Scatter(
                    x=mm_signal.index,
                    y=mm_signal,
                    name = "Signal",
                    line = dict(color = '#B22222'),
                    opacity = 1)

trace_candles = go.Candlestick(x=msft.index,
                    open=msft.Open,
                    high=msft.High,
                    low=msft.Low,
                    close=msft.Close)

fig = tools.make_subplots(rows=2, cols=1)

fig.append_trace(trace_candles, 1, 1)
fig.append_trace(trace_macd, 2, 1)
fig.append_trace(trace_signal, 2, 1)

fig['layout']['yaxis2'].update(range=[-3, 5])

py.iplot(fig, filename='macd-line')

## Histograma MACD

In [ ]:
hist_macd = mm_macd - mm_signal

trace_hist_macd = go.Scatter(
                    x=hist_macd.index,
                    y=hist_macd,
                    fill='tozeroy')

trace_candles = go.Candlestick(x=msft.index,
                    open=msft.Open,
                    high=msft.High,
                    low=msft.Low,
                    close=msft.Close)

fig = tools.make_subplots(rows=2, cols=1)

fig.append_trace(trace_candles, 1, 1)
fig.append_trace(trace_hist_macd, 2, 1)

fig['layout']['yaxis2'].update(range=[-2, 2])

py.iplot(fig, filename='hist-macd-line')

# HiLo Activator

Calcular as médias do hilo

In [ ]:
aapl = all_data.loc['AAPL']['2018-01-01':'2018-12-31'].dropna()

aapl_high_avg = aapl.High.rolling(window=8).mean()
aapl_low_avg = aapl.Low.rolling(window=8).mean()

Plotar as médias em conjunto com o candle

In [ ]:
trace_high = go.Scatter(
                x=aapl_high_avg.index,
                y=aapl_high_avg,
                name = "AAPL High Avg",
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_low = go.Scatter(
                x=aapl_low_avg.index,
                y=aapl_low_avg,
                name = "AAPL Low Avg",
                line = dict(color = '#B22222'),
                opacity = 1)

trace = go.Candlestick(x=aapl.index,
                open=aapl.Open,
                high=aapl.High,
                low=aapl.Low,
                close=aapl.Close)
data = [trace_high, trace_low, trace]
py.iplot(data)

In [ ]:
aapl_high = pd.DataFrame(index=aapl.index)
aapl_low = pd.DataFrame(index=aapl.index)

In [ ]:
aapl_high['high'] = np.where(aapl.Close > aapl_high_avg, aapl_low_avg, np.NaN)
aapl_low['low'] = np.where(aapl.Close < aapl_low_avg, aapl_high_avg, np.NaN)

In [ ]:
trace_high = go.Scatter(
                x=aapl_high.index,
                y=aapl_high,
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_low = go.Scatter(
                x=aapl_low.index,
                y=aapl_low,
                line = dict(color = '#B22222'),
                opacity = 1)

trace = go.Candlestick(x=aapl.index,
                open=aapl.Open,
                high=aapl.High,
                low=aapl.Low,
                close=aapl.Close)
data = [trace_high, trace_low, trace]
py.iplot(data)

# Bandas de Bollinger

In [ ]:
window = 20

aapl = all_data.loc['AAPL']['2018-01-01':'2018-12-31'].dropna()

aapl_avg = aapl.Close.rolling(window=window).mean().dropna()
aapl_std = aapl.Close.rolling(window=window).std().dropna()

In [ ]:
aapl_bollinger = pd.DataFrame(index=aapl_avg.index)
aapl_bollinger['mband'] = aapl_avg
aapl_bollinger['uband'] = aapl_avg + aapl_std.apply(lambda x: (x * 2))
aapl_bollinger['lband'] = aapl_avg - aapl_std.apply(lambda x: (x * 2))

In [ ]:
aapl_prices = aapl[aapl_bollinger.index[0]:]

prices = go.Candlestick(x=aapl_prices.index,
                open=aapl_prices.Open,
                high=aapl_prices.High,
                low=aapl_prices.Low,
                name = "Prices",
                close=aapl_prices.Close)

uband = go.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.uband,
                name = "Upper Band",
                line = dict(color = '#17BECF'),
                opacity = 1)

mband = go.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.mband,
                name = "Moving average",
                line = dict(color = '#B22222'),
                opacity = 1)

lband = go.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.lband,
                name = "Lower Band",
                line = dict(color = '#17BECF'),
                opacity = 1)
data = [prices, uband, lband, mband]
py.iplot(data)

# IFR

In [ ]:
window = 14

aapl = all_data.loc['AAPL'].dropna()
aapl = aapl['2018-06-01':'2019-02-22']

ifr = pd.DataFrame(index=aapl.index)

# get the percentual changes between day[i] and day[i-1]
ifr_changes = aapl.Close.diff(1)

# get days with positive profit (gain)
ifr['gain'] = ifr_changes[ifr_changes > 0]

# get days with negative profit (loss)
ifr['loss'] = ifr_changes[ifr_changes < 0] * (-1)

# gain and loss mean
ifr['gainAvg'] = np.NaN
ifr['lossAvg'] = np.NaN

# get the mean for "window" period
ifr.gainAvg[window] = ifr.iloc[0:window].gain.mean()
ifr.lossAvg[window] = ifr.iloc[0:window].loss.mean()

ifr = ifr[window:].fillna(0.0)

# apply the formula
for i in range(1, len(ifr)):
    ifr.gainAvg[i] = (ifr.gainAvg[i-1] * (window -1) + ifr.gain[i]) / window
    ifr.lossAvg[i] = (ifr.lossAvg[i-1] * (window -1) + ifr.loss[i]) / window
        
ifr['value'] = 100 - (100 / (1 + (ifr.gainAvg / ifr.lossAvg )))


In [ ]:
trace_ifr = go.Scatter(
                    x=ifr.index,
                    y=ifr.value,
                    xaxis='x2',
                    yaxis='y2')

trace_candles = go.Candlestick(x=aapl.index,
                    open=aapl.Open,
                    high=aapl.High,
                    low=aapl.Low,
                    close=aapl.Close)

data = [trace_ifr, trace_candles]

layout = go.Layout(
    xaxis=dict(
        domain=[0, 1],
        rangeslider={"visible": False},
    ),
    yaxis=dict(
        domain=[0.75, 1],        
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2',
        range=[0, 100]
        
    ),
        shapes=[
            # Line Horizontal
            {
                'xref': 'x2',
                'yref': 'y2',
                'type': 'line',
                'x0': aapl.index[0],
                'y0': '20',
                'x1': aapl.index[-1],
                'y1': '20',
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2,
                    'dash': 'dashdot',
                },
            },
            {
                'xref': 'x2',
                'yref': 'y2',
                'type': 'line',
                'x0': aapl.index[0],
                'y0': '70',
                'x1': aapl.index[-1],
                'y1': '70',
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2,
                    'dash': 'dashdot',
                },
            }        
        ]        
    
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='ifr-plot')

# OBV

In [ ]:
aapl = all_data.loc['AAPL'].dropna()
aapl = aapl['2018-06-01':'2019-02-22']

obv = pd.DataFrame(index=aapl.index)

# get the percentual changes between day[i] and day[i-1]
obv_changes = aapl.Close - aapl.Open

obv['open'] = aapl.Open
obv['close'] = aapl.Close
obv['volume'] = np.where(obv_changes > 0, aapl.Volume , aapl.Volume * (-1))

# get days with positive profit (gain)
obv['volume_sum'] = obv.volume.cumsum()

In [ ]:
trace_obv = go.Scatter(
                    x=obv.index,
                    y=obv.volume_sum,
                    xaxis='x2',
                    yaxis='y2')

trace_candles = go.Candlestick(x=aapl.index,
                    open=aapl.Open,
                    high=aapl.High,
                    low=aapl.Low,
                    close=aapl.Close)

data = [trace_obv, trace_candles]

layout = go.Layout(
    xaxis=dict(
        domain=[0, 1],
        rangeslider={"visible": False},
    ),
    yaxis=dict(
        domain=[0.75, 1],        
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
        
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
        
    )
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='obv-plot')

# Estratégia média 9

In [ ]:
tickers = ['VALE3.SA']

start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2018, 6, 28)

all_data = get(tickers, start_date, end_date)

In [ ]:

vale3 = all_data.loc['VALE3.SA'].dropna()

vale3['EMA'] = get_ema(9, vale3.Close).EMA

# get the "slope of EMA9 9"
dif = vale3.EMA - vale3.shift(1).EMA

# ema 9 crossed the candle (open/close price) and ema 9 has an ascendent slope  ?
vale3['mark_max'] = np.where((vale3.EMA < vale3.High) & (vale3.EMA > vale3.Low) & (dif > 0), vale3.High, 0)

# ema 9 crossed the candle (open/close price) and ema 9 has an descendent slope  ?
vale3['mark_min'] = np.where((vale3.EMA > vale3.Low) & (vale3.EMA < vale3.High) & (dif < 0), vale3.Low, 0)

# get start point
vale3['buy_start'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).mark_max, np.NaN)
vale3['sell_start'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).mark_min, np.NaN)

# set stop loss
vale3['buy_stop'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).Low, np.NaN)
vale3['sell_stop'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).High, np.NaN)

display(vale3[["buy_start", "buy_stop"]][(vale3.buy_start > 0)])
display(vale3[["sell_start", "sell_stop"]][(vale3.sell_start > 0)])

## Média 9 com duas médias

In [ ]:
tickers = ['VALE3.SA']

start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2018, 6, 28)

all_data = get(tickers, start_date, end_date)

In [ ]:
vale3 = all_data.loc['VALE3.SA'].dropna()

vale3['EMA9'] = get_ema(9, vale3.Close).EMA
vale3['EMA1'] = get_ema(1, vale3.Close).EMA


buy_cross = (vale3.shift(1).EMA1 < vale3.shift(1).EMA9) & (vale3.EMA1 > vale3.shift(1).EMA9)
sell_cross = (vale3.shift(1).EMA1 > vale3.shift(1).EMA9) & (vale3.EMA1 < vale3.shift(1).EMA9)

# ema 9 crossed the candle (open/close price) and ema 9 has an ascendent slope  ?
vale3['mark_max'] = np.where((vale3.EMA9 < vale3.High) & (vale3.EMA9 > vale3.Low) & (buy_cross == True), vale3.High, 0)

# ema 9 crossed the candle (open/close price) and ema 9 has an descendent slope  ?
vale3['mark_min'] = np.where((vale3.EMA9 > vale3.Low) & (vale3.EMA9 < vale3.High) & (sell_cross == True), vale3.Low, 0)

# get start point
vale3['buy_start'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).mark_max, np.NaN)
vale3['sell_start'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).mark_min, np.NaN)

# set stop loss
vale3['buy_stop'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).Low, np.NaN)
vale3['sell_stop'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).High, np.NaN)

display(vale3[["buy_start", "buy_stop"]][(vale3.buy_start > 0)])
display(vale3[["sell_start", "sell_stop"]][(vale3.sell_start > 0)])

In [ ]:
trace_short = go.Scatter(
                x=vale3.index,
                y=vale3.EMA1,
                name = "EMA(1)",
                line = dict(color = '#FF00CF'),
                opacity = 1)

trace_long = go.Scatter(
                x=vale3.index,
                y=vale3.EMA9,
                name = "EMA(9)",
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_candles = go.Candlestick(x=vale3.index,
                    open=vale3.Open,
                    high=vale3.High,
                    low=vale3.Low,
                    close=vale3.Close)

data = [trace_short, trace_long, trace_candles]

fig = go.Figure(data=data)

py.iplot(fig, filename='cross-mm9-plot')

# ADX

In [ ]:
tickers = ['VALE3.SA']

start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2018, 12, 31)

data = get(tickers, start_date, end_date).loc['VALE3.SA'].dropna()


data_ant = data.shift(1)

# intermediate dataset, whose purpose is to find out the TR value
tr = pd.DataFrame([
        data.High - data.Low, 
        np.absolute(data.High - data_ant.Close), 
        np.absolute(data.Low - data_ant.Close)])

# to facilitate our calculation, we have to transpose the matrix, 
tr = tr.transpose()

# get the max value of each row
tr= tr.max(axis=1)

# assign TR to our dataset
data['tr'] = tr


# get +DM
dm_plus = np.where((data.High - data_ant.High) > (data_ant.Low - data.Low), 
                     (data.High - data_ant.High).apply(
                             lambda x: np.max([x, 0])),
                     0)

# get -DM
dm_minus = np.where((data_ant.Low - data.Low) > (data.High - data_ant.High), 
                     (data_ant.Low - data.Low).apply(
                             lambda x: np.max([x, 0])),
                     0)

# assign DM's to our dataset
data['dmPlus'] = dm_plus
data['dmMinus'] = dm_minus

# remove first line, becouse we won't use it
data = data[1:]

data.head()

In [ ]:
window = 14

data['trAvg'] = np.NaN
data['dmPlusAvg'] = np.NaN
data['dmMinusAvg'] = np.NaN

# sum of 14 periodos for calcaulation of the tr, dmPlus and dmMinus
data.trAvg[window-1] = np.sum(data.iloc[0:window].tr)
data.dmPlusAvg[window-1] = np.sum(data.iloc[0:window].dmPlus)
data.dmMinusAvg[window-1] = np.sum(data.iloc[0:window].dmMinus)

# get data from 14th row until end of the dataset
for i in range(window, len(data)):
    data.trAvg[i] = data.trAvg[i-1] - (data.trAvg[i-1] / window) + data.tr[i]
    data.dmPlusAvg[i] = data.dmPlusAvg[i-1] - (data.dmPlusAvg[i-1] / window) + data.dmPlus[i]
    data.dmMinusAvg[i] = data.dmMinusAvg[i-1] - (data.dmMinusAvg[i-1] / window) + data.dmMinus[i]
    
data = data.dropna()
data.head()    

In [ ]:
# get DI's values
data['diPlus'] = (data.dmPlusAvg / data.trAvg) * 100
data['diMinus'] = (data.dmMinusAvg / data.trAvg) * 100

data.head()

In [ ]:
# get DX value
data['DX'] = (np.absolute(data.diPlus - data.diMinus) / (data.diPlus + data.diMinus)) * 100
data.head()

In [ ]:
data['ADX'] = np.NaN

# average of 14 periods 
data.ADX[window-1] = data.DX[0:window].mean()

# get data from 14th row until end of the dataset
for i in range(window, len(data)):
    data.ADX[i] = (data.ADX[i-1] * (window - 1) + data.DX[i]) / window
    
data = data.dropna()
data.head()

In [ ]:
trace_candles = go.Candlestick(x=data.index,
                    name="VALE3.SA",
                    open=data.Open,
                    high=data.High,
                    low=data.Low,
                    close=data.Close)

trace_di_minus = go.Scatter(
                    x=data.index,
                    y=data.diMinus,
                    name = "-DI",
                    line = dict(color = '#B22222'),
                    opacity = 1)

trace_di_plus = go.Scatter(
                    x=data.index,
                    y=data.diPlus,
                    name = "+DI",
                    line = dict(color = '#17BECF'),
                    opacity = 1)

trace_adx = go.Scatter(
                    x=data.index,
                    y=data.ADX,
                    name = "ADX",
                    line = dict(color = '#9932CC'),
                    opacity = 1)

fig = dict(data=[trace_di_minus, trace_di_plus, trace_adx])

py.iplot(fig, filename='adx-line')

In [ ]:
fig = tools.make_subplots(rows=2, cols=1)

fig.append_trace(trace_candles, 1, 1)
fig.append_trace(trace_di_minus, 2, 1)
fig.append_trace(trace_di_plus, 2, 1)
fig.append_trace(trace_adx, 2, 1)

fig['layout']['yaxis2'].update(range=[0, 100])

py.iplot(fig, filename='adx-line')

# Backtest

In [ ]:
from enum import Enum

class TradeType(Enum):
    NONE = 0
    LONG = 1
    SHORT  = -1

class BackTest:
    def __init__(self, stock, signals, **kwargs):
        self._position = 0
        self._position_opened = False
        self._amount = 0
        self._stock = stock
        self.trades = []
        self._signals = signals
        self._status = None
        self._std_qty_shares = 100
        self._reverse = True
        
        self._show_logs = ("log" in kwargs) and (kwargs["log"])
        
        self.win_trades = 0
        self.loss_trades = 0
        
        
    def _get_win_trades(self, x):
        return x >= 0 
    
    def _get_loss_trades(self, x):
        return x < 0
    
    def _trade(self, position, index):
        """        
        execute a trade
        :position : LONG or SHORT
        :index : index of stock list        
        """
        
        # if has a opened trading, then it will close the position
        if self._position_opened:   
            # set the operating profit
            profit = ((self._std_qty_shares * self._stock.Close[index]) - self._amount)  * (self._position * -1) 
            
            # add proft to trade list
            self.trades.append(profit)            
            
            # close position
            self._position_opened = False            
            
            # log            
            if self._show_logs:
                print(f'Opened[{self._amount}] -> Closed[{self._std_qty_shares * self._stock.Close[index]}] {position.name}')                             
        else:
            # If it hasn't an opened position, open it.
            self._position_opened = True
            self._amount = self._std_qty_shares * self._stock.Close[index]
    
    def start(self): 
        """
            execute the backtest
        """
        for index, item in self._stock.iterrows():
            
            # for long signal (buy)
            if TradeType(self._signals.signal[index]) == TradeType.LONG:
                # Was last signal long signal? go to next iteration
                if TradeType(self._position) == TradeType.LONG: continue

                self._position = self._signals.signal[index]                      
                #if it has a opened short position, then sell
                if self._position_opened:
                    self._trade(TradeType.SHORT, index)
                    # because I'm want to revert my position
                    if self._reverse:
                        self._trade(TradeType.LONG, index)
                else:
                    # ok, I need to open a position
                    self._trade(TradeType.LONG, index)

            # for short signal (sell)
            if TradeType(self._signals.signal[index]) == TradeType.SHORT:
                if TradeType(self._position) == TradeType.SHORT: continue

                self._position = self._signals.signal[index]       

                if self._position_opened:
                    self._trade(TradeType.LONG, index)
                    if self._reverse:
                        self._trade(TradeType.SHORT, index)
                else:
                    self._trade(TradeType.SHORT, index)

        self.win_trades = len(list(filter(self._get_win_trades, self.trades)))
        self.loss_trades = len(list(filter(self._get_loss_trades, self.trades)))

In [ ]:
signal_start_date = '2018-01-01'
signal_end_date = '2018-12-31'
aapl = all_data.loc['AAPL'][signal_start_date:signal_end_date].dropna()

def build_hilo_signal_list(window=5):
    aapl_high_avg = aapl.High.rolling(window=window).mean()
    aapl_low_avg = aapl.Low.rolling(window=window).mean()

    aapl_hilo = pd.DataFrame(index=aapl.index)
    aapl_hilo['high'] = np.where(aapl.Close > aapl_high_avg, 1, 0)
    aapl_hilo['low'] = np.where(aapl.Close < aapl_low_avg, 1, 0)
    aapl_hilo['signal'] = (aapl_hilo.high - aapl_hilo.low)
    
    return aapl_hilo

In [ ]:
signals = build_hilo_signal_list(window=8)
bb = BackTest(aapl, signals, log=True)        
bb.start()
display(sum(bb.trades))

In [ ]:
best_profit = None
print(f'BACKTEST HILO - Start Date: {signal_start_date} / End Date: {signal_end_date}')
print('-' * 100)

for window in range(2, 22):
    signals = build_hilo_signal_list(window=window)
    bb = BackTest(aapl, signals, log=False)        
    bb.start()
    
    profit = sum(bb.trades)
    
    if (best_profit is None) or (best_profit[1] < profit):
        best_profit = (window, profit)
    
    print(f'Profit for window={window}: {profit}')
    
print('-' * 100)    
print(f'Best Profit: window={best_profit[0]}: {best_profit[1]}')    

In [ ]:
signal_start_date = '2018-01-01'
signal_end_date = '2018-12-31'
aapl = all_data.loc['AAPL'][signal_start_date:signal_end_date].dropna()

def build_cross_avg_signal_list(short_window=5, long_window=9):
    aapl_short_avg = aapl.Close.rolling(window=short_window).mean()
    aapl_long_avg = aapl.Close.rolling(window=long_window).mean()

    aapl_cross_avg = pd.DataFrame(index=aapl.index)
    aapl_cross_avg['signal'] = np.where(aapl_short_avg > aapl_long_avg, 1, -1)
    
    return aapl_cross_avg

In [ ]:
signals = build_cross_avg_signal_list(short_window=5, long_window=9)
bb = BackTest(aapl, signals, log=False)        
bb.start()
    
print(sum(bb.trades))
display(bb.win_trades)
display(bb.loss_trades)

In [ ]:
best_profit = None
print(f'BACKTEST CROSS AVG - Start Date: {signal_start_date} / End Date: {signal_end_date}')
print('-' * 100)

for short_window in range(2, 22):
    for long_window in range(2, 22):
        signals = build_cross_avg_signal_list(short_window=short_window, long_window=long_window)
        bb = BackTest(aapl, signals, log=False)        
        bb.start()

        profit = sum(bb.trades)

        if (best_profit is None) or (best_profit[1] < profit):
            best_profit = ((short_window,long_window), profit)

        print(f'Profit for window={short_window}/{long_window}: {profit}')
    
print('-' * 100)    
print(f'Best Profit: window={best_profit[0]}: {best_profit[1]}')    

In [ ]:
buy = float(100 * aapl.head(1).Close)
sell = float(100 * aapl.tail(1).Close)

print(f'BACKTEST BUY AND HOLD - Start Date: {signal_start_date} / End Date: {signal_end_date}')
print('-' * 100)

print(f'Profit: {buy - sell}')

# Script para apagar os gráficos gerados no plotly

In [ ]:
import json
import requests
from requests.auth import HTTPBasicAuth

username = plotly_username
api_key = plotly_key
headers = {'Plotly-Client-Platform': 'python'}

auth = HTTPBasicAuth(username, api_key)

def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next'] 
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page
        
def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)


permanently_delete_files(username, filetype_to_delete='plot')
permanently_delete_files(username, filetype_to_delete='grid')

In [ ]:
aapl = all_data.loc['AAPL']['2018-01-01':'2018-12-31'].dropna()